In [3]:
from ipynb_settings import *

# NPC Control + Levels

Мы умеем создавать `NPC`, причем сколько угодно. Поэтому дава создавдим класс, который будет отвечать за то, как эти `NPC` располагаются на экране/их цвет/скорость и т.п.

- Создадим файл `npc_control` и реализуем класс `NpcControl`

In [4]:
class NcpControl:
    def __init__(self, game):
        self.game = game
        self.npc_list = []
        # запустим функцию, в которой добавим всех нужных нам NPC
        self.add_npc()
    
    def add_npc(self):
        pass
    
    def update(self):
        pass
    
    def draw(self):
        pass

- Реализуем функцию `add_npc`, `update`, `draw`

In [5]:
#! Импортируем класс NPC в npc_control
from npc import *

class NpcControl:
    def __init__(self, game):
        self.game = game
        self.npc_list = []
        self.add_npc()
        
    def add_npc(self):
        self.npc_list.append(NPC(game=self.game, pos=(11.5, 7.5)))
        self.npc_list.append(NPC(game=self.game, pos=(6.5, 7.5)))
        self.npc_list.append(NPC(game=self.game, pos=(9.5, 1.5)))
        self.npc_list.append(NPC(game=self.game, pos=(13, 3)))
    
    def update(self):
        #! Проходимся по всему списку npc_list и запускаем update каждого
        for npc in self.npc_list:
            npc.update()
    
    def draw(self):
        for npc in self.npc_list:
            npc.draw()

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


- Теперь обновим класс `Game` в `main.py`

In [ ]:
# main.py
import pygame as pg
import sys
from settings import *
from map import *
from player import *
#! убираем import npc, потому что мы запускаем npc с помощью npc_control
#? - from npc import *

#! импортируем npc_control
from npc_control import *
from path_finding import *



class Game:
    def __init__(self) -> None:
        pg.init()
        self.screen = pg.display.set_mode(size=RES)
        self.clock = pg.time.Clock()
        self.new_game()

    def new_game(self):
        self.delta_time = self.clock.tick(FPS)
        self.map = Map(game=self)
        self.player = Player(game=self)
        #! Убираем добавление npc, потому что будем делать с помощью NpcControl
        #? - self.npc = NPC(game=self, pos=(8, 5))
        
        #! Добавляем переменную npc_control
        self.npc_control = NpcControl(game=self)
        self.path_finding = PathFinding(game=self)

    def update(self):
        pg.display.flip()
        self.delta_time = self.clock.tick(FPS)
        self.player.update()
        #! Убираем update npc
        #? - self.npc.update()
        self.npc_control.update()

    def draw(self):
        self.screen.fill('black')
        self.map.draw()
        self.player.draw()
        #! Убираем draw npc
        #? - self.npc.draw()
        self.npc_control.draw()

    def check_event(self):
        for event in pg.event.get():
            if event.type == pg.QUIT or (event.type == pg.KEYDOWN and event.key == pg.K_ESCAPE):
                pg.quit()
                sys.exit()

    def run(self):
        while True:
            self.check_event()
            self.draw()
            self.update()


if __name__ == '__main__':
    game = Game()
    game.run()


Проблема: два `NPC` объединяются в один
1. Хранить все позиции `NPC`
2. Исправить `npc`
3. Исправить `path_finding`

In [ ]:
from npc import *

class NpcControl:
    def __init__(self, game):
        self.game = game
        self.npc_list = []
        #! создаем список позиций npc
        self.npc_positions = {}
        self.add_npc()
        
    def add_npc(self):
        self.npc_list.append(NPC(game=self.game, pos=(11.5, 7.5)))
        self.npc_list.append(NPC(game=self.game, pos=(6.5, 7.5)))
        self.npc_list.append(NPC(game=self.game, pos=(9.5, 1.5)))
        self.npc_list.append(NPC(game=self.game, pos=(13, 3)))
    
    def update(self):
        for npc in self.npc_list:
            npc.update()
        
        #! сохраняем в множество позиции npc
        self.npc_positions.clear()
        for npc in self.npc_list:
            self.npc_positions.add(npc.map_pos)
    
    
    def draw(self):
        for npc in self.npc_list:
            npc.draw()

- Меняем `npc`. Добавляем проверку, что никто не стоит на следующей позиции в функцию `move`:

```python 
if next_pos not in self.game.npc_control.npc_positions:
    self.check_wall_collision(dx, dy)
```

In [ ]:

import pygame as pg
import math
from settings import *


class NPC():
    def __init__(self, game, pos):
        self.x, self.y = pos
        self.game = game
        self.player = game.player
        self.theta = 0
        self.speed = SPEED_NPC

    #! Добавили функцию, которая возвращает положение npc в мире
    @property
    def map_pos(self):
        return int(self.x), int(self.y)

    def draw(self):
        pg.draw.circle(self.game.screen, self.color,
                       (100 * self.x, 100 * self.y), 15)
        pg.draw.line(self.game.screen, self.color, (self.x * 100 + 14.5 * math.sin(self.theta), self.y * 100 - 14.5 * math.cos(self.theta)),
                     (self.x * 100 - 35 * math.cos(self.theta),
                     self.y * 100 - 35 * math.sin(self.theta)), 2)
        pg.draw.line(self.game.screen, self.color, (self.x * 100 - 14.5 * math.sin(self.theta), self.y * 100 + 14.5 * math.cos(self.theta)),
                     (self.x * 100 - 35 * math.cos(self.theta),
                     self.y * 100 - 35 * math.sin(self.theta)), 2)

    def look(self):
        dx = self.x - self.player.x
        dy = self.y - self.player.y
        self.dx, self.dy = dx, dy
        self.theta = math.atan2(dy, dx)

        delta = self.theta - self.player.angle
        if (dx > 0 and self.player.angle > math.pi) or (dx < 0 and dy < 0):
            delta += math.tau

        self.dist = math.hypot(dx, dy)
        self.norm_dist = self.dist * math.cos(delta)

    def check_wall(self, x, y):
        return (x, y) not in self.game.map.world_map

    def check_wall_collision(self, dx, dy):
        if self.check_wall(int(self.x + dx), int(self.y)):
            self.x += dx
        if self.check_wall(int(self.x), int(self.y + dy)):
            self.y += dy

    def move(self):
        next_pos = self.game.path_finding.get_path(
            self.map_pos, self.game.player.map_pos)
        next_x, next_y = next_pos

        angle = math.atan2(next_y + 0.5 - self.y, next_x + 0.5 - self.x)
        dx = math.cos(angle) * self.speed * self.game.delta_time
        dy = math.sin(angle) * self.speed * self.game.delta_time

        #! Добавляем проверку, что на следующей клетке никто не стоит
        if next_pos not in self.game.npc_control.npc_positions:
            self.check_wall_collision(dx, dy)

    def ray_cast_player_npc(self):
        if self.game.player.map_pos == self.map_pos:
            return True

        wall_dist_v, wall_dist_h = 0, 0
        player_dist_v, player_dist_h = 0, 0

        ox, oy = self.game.player.pos
        x_map, y_map = self.game.player.map_pos

        ray_angle = self.theta

        sin_a = math.sin(ray_angle)
        cos_a = math.cos(ray_angle)

        # horizontals
        y_hor, dy = (y_map + 1, 1) if sin_a > 0 else (y_map - 1e-6, -1)

        depth_hor = (y_hor - oy) / sin_a
        x_hor = ox + depth_hor * cos_a

        delta_depth = dy / sin_a
        dx = delta_depth * cos_a

        for i in range(MAX_DEPTH):
            tile_hor = int(x_hor), int(y_hor)
            if tile_hor == self.map_pos:
                player_dist_h = depth_hor
                break
            if tile_hor in self.game.map.world_map:
                wall_dist_h = depth_hor
                break
            x_hor += dx
            y_hor += dy
            depth_hor += delta_depth

        # verticals
        x_vert, dx = (x_map + 1, 1) if cos_a > 0 else (x_map - 1e-6, -1)

        depth_vert = (x_vert - ox) / cos_a
        y_vert = oy + depth_vert * sin_a

        delta_depth = dx / cos_a
        dy = delta_depth * sin_a

        for i in range(MAX_DEPTH):
            tile_vert = int(x_vert), int(y_vert)
            if tile_vert == self.map_pos:
                player_dist_v = depth_vert
                break
            if tile_vert in self.game.map.world_map:
                wall_dist_v = depth_vert
                break
            x_vert += dx
            y_vert += dy
            depth_vert += delta_depth

        player_dist = max(player_dist_v, player_dist_h)
        wall_dist = max(wall_dist_v, wall_dist_h)

        if 0 < player_dist < wall_dist or not wall_dist:
            return True
        return False

    #! Дописали run_logic
    def run_logic(self):
        self.ray_cast_value = self.ray_cast_player_npc()
        if self.ray_cast_value == True:
            self.move()

    def update(self):
        self.look()
        self.run_logic()


- Меняем `path_finding`

In [ ]:
# path_finding.py

from collections import deque


class PathFinding:
    def __init__(self, game):
        self.game = game
        self.map = game.map.mini_map
        self.ways = [-1, 0], [0, -1], [1, 0], [0, 1], [-1, -1], [1, -1], [1, 1], [-1, 1]
        self.graph = {}
        self.get_graph()

    def get_path(self, start, goal):
        self.visited = self.bfs(start, goal, self.graph)
        path = [goal]
        step = self.visited.get(goal, start)

        while step and step != start:
            path.append(step)
            step = self.visited[step]
        return path[-1]

    def bfs(self, start, goal, graph):
        queue = deque([start])
        visited = {start: None}

        while queue:
            cur_node = queue.popleft()
            if cur_node == goal:
                break
            
            #! Ура, нам понадобились эти строчки
            next_nodes = graph[cur_node]
            for next_node in next_nodes:
                if next_node not in visited and next_node not in self.game.npc_control.npc_positions:
                    queue.append(next_node)
                    visited[next_node] = cur_node
            
        return visited

    def get_next_nodes(self, x, y):
        return [(x + dx, y + dy) for dx, dy in self.ways if (x + dx, y + dy) not in self.game.map.world_map]

    def get_graph(self):
        for y, row in enumerate(self.map):
            for x, col in enumerate(row):
                if not col:
                    self.graph[(x, y)] = self.graph.get((x, y), []) + self.get_next_nodes(x, y)



- Теперь давайте сделаем, чтобы мы могли контролировать скорость/цвет и другие параметры `npc` при создании копии. Для этого надо просто изменить функцию `__init__` для создания экземпляра

In [ ]:

import pygame as pg
import math
from settings import *


class NPC():
    #! Добавили возможность изменять скорость и цвет, поставили значения по умолчанию
    def __init__(self, game, pos, color='red', speed=SPEED_NPC):
        self.x, self.y = pos
        self.color = color
        self.game = game
        self.player = game.player
        self.theta = 0
        self.speed = speed

    #! Добавили функцию, которая возвращает положение npc в мире
    @property
    def map_pos(self):
        return int(self.x), int(self.y)

    def draw(self):
        pg.draw.circle(self.game.screen, self.color,
                       (100 * self.x, 100 * self.y), 15)
        pg.draw.line(self.game.screen, self.color, (self.x * 100 + 14.5 * math.sin(self.theta), self.y * 100 - 14.5 * math.cos(self.theta)),
                     (self.x * 100 - 35 * math.cos(self.theta),
                     self.y * 100 - 35 * math.sin(self.theta)), 2)
        pg.draw.line(self.game.screen, self.color, (self.x * 100 - 14.5 * math.sin(self.theta), self.y * 100 + 14.5 * math.cos(self.theta)),
                     (self.x * 100 - 35 * math.cos(self.theta),
                     self.y * 100 - 35 * math.sin(self.theta)), 2)

    def look(self):
        dx = self.x - self.player.x
        dy = self.y - self.player.y
        self.dx, self.dy = dx, dy
        self.theta = math.atan2(dy, dx)

        delta = self.theta - self.player.angle
        if (dx > 0 and self.player.angle > math.pi) or (dx < 0 and dy < 0):
            delta += math.tau

        self.dist = math.hypot(dx, dy)
        self.norm_dist = self.dist * math.cos(delta)

    def check_wall(self, x, y):
        return (x, y) not in self.game.map.world_map

    def check_wall_collision(self, dx, dy):
        if self.check_wall(int(self.x + dx), int(self.y)):
            self.x += dx
        if self.check_wall(int(self.x), int(self.y + dy)):
            self.y += dy

    def move(self):
        next_pos = self.game.path_finding.get_path(
            self.map_pos, self.game.player.map_pos)
        next_x, next_y = next_pos

        angle = math.atan2(next_y + 0.5 - self.y, next_x + 0.5 - self.x)
        dx = math.cos(angle) * self.speed * self.game.delta_time
        dy = math.sin(angle) * self.speed * self.game.delta_time

        #! Добавляем проверку, что на следующей клетке никто не стоит
        if next_pos not in self.game.npc_control.npc_positions:
            self.check_wall_collision(dx, dy)

    def ray_cast_player_npc(self):
        if self.game.player.map_pos == self.map_pos:
            return True

        wall_dist_v, wall_dist_h = 0, 0
        player_dist_v, player_dist_h = 0, 0

        ox, oy = self.game.player.pos
        x_map, y_map = self.game.player.map_pos

        ray_angle = self.theta

        sin_a = math.sin(ray_angle)
        cos_a = math.cos(ray_angle)

        # horizontals
        y_hor, dy = (y_map + 1, 1) if sin_a > 0 else (y_map - 1e-6, -1)

        depth_hor = (y_hor - oy) / sin_a
        x_hor = ox + depth_hor * cos_a

        delta_depth = dy / sin_a
        dx = delta_depth * cos_a

        for i in range(MAX_DEPTH):
            tile_hor = int(x_hor), int(y_hor)
            if tile_hor == self.map_pos:
                player_dist_h = depth_hor
                break
            if tile_hor in self.game.map.world_map:
                wall_dist_h = depth_hor
                break
            x_hor += dx
            y_hor += dy
            depth_hor += delta_depth

        # verticals
        x_vert, dx = (x_map + 1, 1) if cos_a > 0 else (x_map - 1e-6, -1)

        depth_vert = (x_vert - ox) / cos_a
        y_vert = oy + depth_vert * sin_a

        delta_depth = dx / cos_a
        dy = delta_depth * sin_a

        for i in range(MAX_DEPTH):
            tile_vert = int(x_vert), int(y_vert)
            if tile_vert == self.map_pos:
                player_dist_v = depth_vert
                break
            if tile_vert in self.game.map.world_map:
                wall_dist_v = depth_vert
                break
            x_vert += dx
            y_vert += dy
            depth_vert += delta_depth

        player_dist = max(player_dist_v, player_dist_h)
        wall_dist = max(wall_dist_v, wall_dist_h)

        if 0 < player_dist < wall_dist or not wall_dist:
            return True
        return False

    #! Дописали run_logic
    def run_logic(self):
        self.ray_cast_value = self.ray_cast_player_npc()
        if self.ray_cast_value == True:
            self.move()

    def update(self):
        self.look()
        self.run_logic()
